In [4]:
import sympy as s

In [7]:
i1,i2,i3,i4,i5,i6 = s.symbols(" ".join([f"x_{{i{i}}}" for i in range(6)]))
j1,j2,j3,j4,j5,j6 = s.symbols(" ".join([f"x_{{j{i}}}" for i in range(6)]))
z1,z2,z3,z4,z5,z6,z7,z8,z9,z10,z11,z12 = s.symbols(" ".join([f"z_{{{i}}}" for i in range(12)]))
theta_i, theta_j, theta_z = s.symbols(r"\theta_i \theta_j \theta_z")
Zij = s.Matrix([
    [z1,z2,z3,z10],
    [z4,z5,z6,z11],
    [z7,z8,z9,z12],
    [0,0,0,1]
])

theta_i

\theta_i

In [8]:
def cross(a,b,c):
    return s.Matrix([
        [0,-c,b],
        [c,0,-a],
        [-b,a,0]
    ])

def exp(x1,x2,x3,x4,x5,x6):
    theta = s.sqrt(x4*x4 + x5*x5 + x6*x6)
    a = s.sin(theta)/theta
    b = (1 - s.cos(theta))/(theta*theta)
    c = (theta - s.sin(theta))/(theta**3)
    w = cross(x4,x5,x6)
    R = s.eye(3) + a*w + b*w@w

    t = (s.eye(3) + b*w + c*w@w)@s.Matrix([[x1], [x2], [x3]])
    
    mat = s.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def log(T):
    theta = s.acos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return T[0, 3], T[1, 3], T[2, 3], 0, 0, 0

    a = s.sin(theta)/theta
    b = (1 - s.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = cross(x4,x5,x6)
    u = (s.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return u[0], u[1], u[2], x4, x5, x6

log(exp(0.1,0.2,0.3,0.4,0.5,0.6))

(0.100000000000000,
 0.200000000000000,
 0.300000000000000,
 0.400000000000000,
 0.500000000000000,
 0.600000000000000)

In [9]:
def invert(T):
    R = T[:3, :3]
    t = T[:3, 3]
    
    mat = s.eye(4)
    mat[:3, :3] = R.T
    mat[:3, 3] = -R.T@t

    return mat

In [10]:
Xi = exp(i1,i2,i3,i4,i5,i6)
Xj = exp(j1,j2,j3,j4,j5,j6)
Zij = exp(z1,z2,z3,z4,z5,z6)

Xi = Xi.subs(s.sqrt(i4*i4 + i5*i5 + i6*i6), theta_i)
Xj = Xj.subs(s.sqrt(j4*j4 + j5*j5 + j6*j6), theta_j)
Zij = Zij.subs(s.sqrt(z4*z4 + z5*z5 + z6*z6), theta_z)

Xi

Matrix([
[1 - x_{i4}**2*(1 - cos(\theta_i))/\theta_i**2 - x_{i5}**2*(1 - cos(\theta_i))/\theta_i**2,            -x_{i5}*sin(\theta_i)/\theta_i + x_{i3}*x_{i4}*(1 - cos(\theta_i))/\theta_i**2,             x_{i4}*sin(\theta_i)/\theta_i + x_{i3}*x_{i5}*(1 - cos(\theta_i))/\theta_i**2, x_{i0}*(1 - x_{i4}**2*(\theta_i - sin(\theta_i))/\theta_i**3 - x_{i5}**2*(\theta_i - sin(\theta_i))/\theta_i**3) + x_{i1}*(-x_{i5}*(1 - cos(\theta_i))/\theta_i**2 + x_{i3}*x_{i4}*(\theta_i - sin(\theta_i))/\theta_i**3) + x_{i2}*(x_{i4}*(1 - cos(\theta_i))/\theta_i**2 + x_{i3}*x_{i5}*(\theta_i - sin(\theta_i))/\theta_i**3)],
[            x_{i5}*sin(\theta_i)/\theta_i + x_{i3}*x_{i4}*(1 - cos(\theta_i))/\theta_i**2, 1 - x_{i3}**2*(1 - cos(\theta_i))/\theta_i**2 - x_{i5}**2*(1 - cos(\theta_i))/\theta_i**2,            -x_{i3}*sin(\theta_i)/\theta_i + x_{i4}*x_{i5}*(1 - cos(\theta_i))/\theta_i**2, x_{i0}*(x_{i5}*(1 - cos(\theta_i))/\theta_i**2 + x_{i3}*x_{i4}*(\theta_i - sin(\theta_i))/\theta_i**3) + x_{i1}*(1 - 

In [11]:
e = log(invert(Zij)@invert(Xi)@Xj)
e[4]

((-x_{j3}*sin(\theta_j)/\theta_j + x_{j4}*x_{j5}*(1 - cos(\theta_j))/\theta_j**2)*((-x_{i3}*sin(\theta_i)/\theta_i + x_{i4}*x_{i5}*(1 - cos(\theta_i))/\theta_i**2)*(-z_{4}*sin(\theta_z)/\theta_z + z_{3}*z_{5}*(1 - cos(\theta_z))/\theta_z**2) + (x_{i5}*sin(\theta_i)/\theta_i + x_{i3}*x_{i4}*(1 - cos(\theta_i))/\theta_i**2)*(1 - z_{4}**2*(1 - cos(\theta_z))/\theta_z**2 - z_{5}**2*(1 - cos(\theta_z))/\theta_z**2) + (z_{5}*sin(\theta_z)/\theta_z + z_{3}*z_{4}*(1 - cos(\theta_z))/\theta_z**2)*(1 - x_{i3}**2*(1 - cos(\theta_i))/\theta_i**2 - x_{i5}**2*(1 - cos(\theta_i))/\theta_i**2)) - (-x_{j4}*sin(\theta_j)/\theta_j + x_{j3}*x_{j5}*(1 - cos(\theta_j))/\theta_j**2)*((x_{i3}*sin(\theta_i)/\theta_i + x_{i4}*x_{i5}*(1 - cos(\theta_i))/\theta_i**2)*(-z_{3}*sin(\theta_z)/\theta_z + z_{4}*z_{5}*(1 - cos(\theta_z))/\theta_z**2) + (-x_{i4}*sin(\theta_i)/\theta_i + x_{i3}*x_{i5}*(1 - cos(\theta_i))/\theta_i**2)*(z_{4}*sin(\theta_z)/\theta_z + z_{3}*z_{5}*(1 - cos(\theta_z))/\theta_z**2) + (1 - x_{i3

# Check simplification

In [12]:
Xi = exp(i1,i2,i3,i4,i5,i6)
Xj = exp(j1,j2,j3,j4,j5,j6)
Zij = exp(z1,z2,z3,z4,z5,z6)

In [13]:
RXi = Xi[:3,:3]
RXj = Xj[:3,:3]
RZij = Zij[:3,:3]
tXi = Xi[:3, 3]
tXj = Xj[:3, 3]
tZij = Zij[:3, 3]

In [14]:
res1 = invert(Zij)@invert(Xi)@Xj

In [15]:
res2 = s.eye(4)
res2[:3,:3] = RZij.T@RXi.T@RXj
res2[:3, 3] = RZij.T@(RXi.T@(tXj - tXi) - tZij)

Pretty sure that does give me the correct result

$$Z_{ij}^{-1}X_i^{-1}X_j \equiv [R_z^TR_i^TR_j | R_z^T(R_i^T(t_j - t_i) - t_z)]$$

However how I take the derivative of that with the log function I'm not entirely sure. I guess I can pass it through the log function and then take the elementwise derivative but that sounds rather messy to say the least. Honestly this whole thing is getting rather messy. It would be really nice if there was a way to get take the derivative of the log function and just chain rule it.

[This answer](https://math.stackexchange.com/questions/723262/explicit-proof-of-the-derivative-of-a-matrix-logarithm) suggests that the derivative of the matrix log function is:
$$X'(x)X^{-1}$$

Which I'm going to try use to solve this silly little jacobian. I'm not entirely convinced that its the right way to do it but I'm going to give it a shot anyway. Actually wait - if we look at the dimensions of it - I'm going to end up with a $4\times4$ matrix which definitely isnt the $6\times1$ vector that I would want from this. Damn. Thats not going to work at all. Back to the drawing board I guess. 

$\ominus$

In [16]:
import numpy as np

def npcross(x):
    return np.array([
        [0,-x[2],x[1]],
        [x[2],0,-x[0]],
        [-x[1],x[0],0]
    ])

def npexp(x):
    theta = np.linalg.norm(x[3:])
    if theta == 0:
        R = np.eye(3)
        t = x[:3]
    else:
        a = np.sin(theta)/theta
        b = (1 - np.cos(theta))/(theta*theta)
        c = (theta - np.sin(theta))/(theta**3)
        w = npcross(x[3:])
        R = np.eye(3) + a*w + b*w@w

        t = (np.eye(3) + b*w + c*w@w)@x[:3]
    
    mat = np.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def nplog(T):
    theta = np.arccos((np.trace(T) - 2) / 2)
    # theta = np.arccos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return T[0, 3], T[1, 3], T[2, 3], 0, 0, 0

    a = np.sin(theta)/theta
    b = (1 - np.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = npcross(np.array([x4,x5,x6]))
    u = (np.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return u[0], u[1], u[2], x4, x5, x6

np.isclose(nplog(npexp(np.array([0.1,0.2,0.3,0.4,0.5,0.6]))), np.array([0.1,0.2,0.3,0.4,0.5,0.6]))

array([ True,  True,  True,  True,  True,  True])

In [17]:
def np_adjoint(X):
    """X is a 4x4 matrix not a 6 Vector"""
    mat = np.zeros((6,6))
    mat[:3,:3] = X[:3,:3]
    mat[3:,3:] = X[:3,:3]
    mat[:3,3:] = npcross(X[:3,3])@X[:3,:3]

    return mat.T


In [18]:
def np_error(Z, Xi, Xj):
    return nplog(np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@npexp(Xj))

def np_exp_error(Z, Xi, Xj):
    return np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@npexp(Xj)


In [19]:
def Je_numeric_Xi(Z, Xi, Xj, eps=1E-4, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi)@npexp(g_i*eps))@npexp(Xj) for g_i in generators]

    cols = (1/eps)*np.array([nplog(error_inv@gen_error_i) for gen_error_i in gen_error])

    return cols

a = np.array([0.1,0.2,0.3,0.4,0.5,0.6])
with np.printoptions(precision=4, suppress=True):
    print(Je_numeric_Xi(a,a,a))

[[-1. -0. -0.  0.  0.  0.]
 [ 0. -1. -0.  0.  0.  0.]
 [ 0. -0. -1.  0.  0.  0.]
 [ 0. -0. -0. -1.  0.  0.]
 [-0. -0. -0.  0. -1.  0.]
 [ 0. -0. -0.  0. -0. -1.]]


In [20]:
def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = npexp(Xi), npexp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@npcross(eXj[:3,3])
    # print("A: ")
    # print(A)

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = npcross(eXi[:3, 3])@eXi[:3,:3]
    # print("\n\nB: ")
    # print(B)

    return -(A@B).T


with np.printoptions(precision=4, suppress=True):
    vals = (np.random.random(6),np.random.random(6),np.random.random(6))
    res_analytic = Je_analytic_Xi(*vals[:])
    res_numeric = Je_numeric_Xi(*vals[:], eps=1E-7)
    print("\n\nResult analytic: ")
    print(res_analytic)
    print("\n\nResult numeric: ")
    print(res_numeric)
    print(np.all(np.isclose(res_analytic, res_numeric)))



Result analytic: 
[[-0.9291  0.3692  0.0221 -0.     -0.     -0.    ]
 [-0.3642 -0.9237  0.1191 -0.     -0.     -0.    ]
 [-0.0644 -0.1026 -0.9926 -0.     -0.     -0.    ]
 [-0.1224 -0.2931 -0.249  -0.9291  0.3692  0.0221]
 [ 0.2682 -0.101   0.0371 -0.3642 -0.9237  0.1191]
 [ 0.249  -0.1456 -0.0011 -0.0644 -0.1026 -0.9926]]


Result numeric: 
[[-0.9291  0.3692  0.0221  0.      0.      0.    ]
 [-0.3642 -0.9237  0.1191  0.      0.      0.    ]
 [-0.0644 -0.1026 -0.9926  0.      0.      0.    ]
 [-0.1224 -0.2931 -0.249  -0.9291  0.3692  0.0221]
 [ 0.2682 -0.101   0.0371 -0.3642 -0.9237  0.1191]
 [ 0.249  -0.1456 -0.0011 -0.0644 -0.1026 -0.9926]]
True


In [60]:
def Je_numeric_Xj(Z, Xi, Xj, eps=1E-4, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@(npexp(Xj)@npexp(g_i*eps)) for g_i in generators]

    cols = (1/eps)*np.array([nplog(error_inv@gen_error_i) for gen_error_i in gen_error])

    return cols

a = np.array([0.1,0.2,0.3,0.4,0.5,0.6])
with np.printoptions(precision=4, suppress=True):
    vals = (np.random.random(6),np.random.random(6),np.random.random(6))
    print(Je_numeric_Xj(*vals))

[[ 1. -0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0. -0.  1.  0.  0.  0.]
 [ 0. -0. -0.  1. -0.  0.]
 [ 0. -0.  0.  0.  1.  0.]
 [ 0. -0.  0. -0.  0.  1.]]


## In vector space
The above, I think, gives the results of the jacobian in the se(3) space rather than the vector space of the exponential coordinates. So what I think I need to do is pre-multiply by the adjoint of the whole error function but I'm not entirely convinced thats the case.

In [71]:
def Je_numeric_Xi(Z, Xi, Xj, eps=1E-4, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi)@npexp(g_i*eps))@npexp(Xj) for g_i in generators]
    # gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi + g_i*eps))@npexp(Xj) for g_i in generators]

    cols = (1/eps)*np.array([nplog(error_inv@gen_error_i) for gen_error_i in gen_error])

    return cols

def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = npexp(Xi), npexp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@npcross(eXj[:3,3])
    # print("A: ")
    # print(A)

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = npcross(eXi[:3, 3])@eXi[:3,:3]
    # print("\n\nB: ")
    # print(B)

    adj = np_adjoint(np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@npexp(Xj))

    return -(A@B).T


with np.printoptions(precision=4, suppress=True):
    vals = (np.random.random(6),np.random.random(6),np.random.random(6))
    res_analytic = Je_analytic_Xi(*vals[:])
    res_numeric = Je_numeric_Xi(*vals[:], eps=1E-7)
    print("\n\nResult analytic: ")
    print(res_analytic)
    print("\n\nResult numeric: ")
    print(res_numeric)
    # print("\n\nDifference: ")
    # print(res_numeric@np.linalg.inv(res_analytic))
    # print("\n\nAdjoint: ")
    # print(np_adjoint(np.linalg.inv(npexp(vals[0]))@np.linalg.inv(npexp(vals[1]))@npexp(vals[2])))
    # print(np_adjoint(np.linalg.inv(npexp(vals[1]))))

    print(np.isclose(res_analytic, res_numeric))



Result analytic: 
[[-0.9037  0.0896 -0.4188 -0.     -0.     -0.    ]
 [-0.1073 -0.994   0.0189 -0.     -0.     -0.    ]
 [ 0.4146 -0.062  -0.9079 -0.     -0.     -0.    ]
 [-0.3432 -0.1331  0.7121 -0.9037  0.0896 -0.4188]
 [ 0.2184 -0.0243 -0.0406 -0.1073 -0.994   0.0189]
 [-0.6915  0.1981 -0.3293  0.4146 -0.062  -0.9079]]


Result numeric: 
[[-0.9037  0.0896 -0.4188  0.      0.      0.    ]
 [-0.1073 -0.994   0.0189  0.      0.      0.    ]
 [ 0.4146 -0.062  -0.9079  0.      0.      0.    ]
 [-0.3432 -0.1331  0.7121 -0.9037  0.0896 -0.4188]
 [ 0.2184 -0.0243 -0.0406 -0.1073 -0.994   0.0189]
 [-0.6915  0.1981 -0.3293  0.4146 -0.062  -0.9079]]
[[ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]]
